## Mask-RCNN for Chest X-ray Abnormalities Detection

Thank you for these excellent notebooks:

1. [Mask-RCNN and Medical Transfer Learning SIIM-ACR](https://www.kaggle.com/hmendonca/mask-rcnn-and-medical-transfer-learning-siim-acr)
2. [MaskRCNN for Chest X-ray Anomaly Detection](https://www.kaggle.com/frlemarchand/maskrcnn-for-chest-x-ray-anomaly-detection)
3. [Segmenting Nuclei in Microscopy Images](https://github.com/matterport/Mask_RCNN/blob/master/samples/nucleus/nucleus.py) 

This notebook aims to inspect the trained model and the stage of submission. The model was trained with backbone ResNet101 + augmentation data + stratied k-fold.
I look forward to receiving feedback from you on how to boost the efficiency of the MaskRCNN model. 

Thank you.

In [ ]:
import os
os.makedirs("../working/mrcnn")
os.makedirs("../working/test_img")

In [ ]:
from shutil import copyfile
copyfile(src = "../input/mrcnn-tf2/config.py", dst = "../working/mrcnn/config.py")
copyfile(src = "../input/mrcnn-tf2/model.py", dst = "../working/mrcnn/model.py")
copyfile(src = "../input/mrcnn-tf2/visualize.py", dst = "../working/mrcnn/visualize.py")
copyfile(src = "../input/mrcnn-tf2/utils.py", dst = "../working/mrcnn/utils.py")
copyfile(src = "../input/mrcnn-tf2/parallel_model.py", dst = "../working/mrcnn/parallel_model.py")

In [ ]:
# !rm -rf ../working/mrcnn/

In [ ]:
import os
import random
from random import randint

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import cv2
from tqdm import tqdm
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from skimage import exposure
from imgaug import augmenters as iaa

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import tensorflow as tf
tf.__version__

from tensorflow.python.client import device_lib
print("GPU sample processing: ")
print(device_lib.list_local_devices())

In [ ]:
DATA_DIR = "../input/vinbigdata-chest-xray-abnormalities-detection"
TRAIN_DIR = os.path.join(DATA_DIR, "train")
TEST_DIR = os.path.join(DATA_DIR, "test")
TRAIN_CSV_DIR = os.path.join(DATA_DIR, "train.csv")
MASKRCNN_CSV_DIR = '../input/trainingdf/sample_df.csv'

PREPROCESSED_TRAINING_IMAGE_FOLDER = "../input/512-jpg/512_jpg/"
TEMP_TEST_FOLDER = "../working/test_img/"

In [ ]:
org_df = pd.read_csv(TRAIN_CSV_DIR)
org_df = org_df.query('class_id != 14')

In [ ]:
training_df = pd.read_csv(MASKRCNN_CSV_DIR, converters ={'EncodedPixels': eval, 'CategoryId': eval})

In [ ]:
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

In [ ]:
NUM_CATS = 14
IMAGE_SIZE = 512

In [ ]:
class DiagnosticConfig(Config):
    NAME = "Diagnostic"
    NUM_CLASSES = NUM_CATS + 1 # +1 for the background class

    GPU_COUNT = 1
    IMAGES_PER_GPU = 10 #That is the maximum with the memory available on kernels

    BACKBONE = 'resnet101'

    IMAGE_MIN_DIM = IMAGE_SIZE
    IMAGE_MAX_DIM = IMAGE_SIZE
    IMAGE_RESIZE_MODE = 'none'

    POST_NMS_ROIS_TRAINING = 250
    POST_NMS_ROIS_INFERENCE = 150
    MAX_GROUNDTRUTH_INSTANCES = 5
    BACKBONE_STRIDES = [4, 8, 16, 32, 64]
    BACKBONESHAPE = (8, 16, 24, 32, 48)
    RPN_ANCHOR_SCALES = (8,16,24,32,48)
    ROI_POSITIVE_RATIO = 0.33
    DETECTION_MAX_INSTANCES = 300
    DETECTION_MIN_CONFIDENCE = 0.7


    STEPS_PER_EPOCH = int(len(training_df)*0.8/IMAGES_PER_GPU)
    VALIDATION_STEPS = int(len(training_df)/IMAGES_PER_GPU)-int(len(training_df)*0.9/IMAGES_PER_GPU)

config = DiagnosticConfig()

In [ ]:
# Create Inference Config
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

In [ ]:
category_list = ["Aortic enlargement", "Atelectasis","Calcification","Cardiomegaly","Consolidation","ILD",
                "Infiltration", "Lung opacity", "Nodule/ Mass","Other lesion","Pleural effusion",
                "Pleural thickening", "Pneumothorax","Pulmonary fibrosis"]
category_list

In [ ]:
# Create Mask RCNN formart dataset
class DiagnosticDataset(utils.Dataset):
    def __init__(self, df):
        super().__init__(self)

        # Add classes
        for i, name in enumerate(category_list):
            self.add_class("diagnostic", i+1, name)

        # Add images
        for i, row in df.iterrows():
            self.add_image("diagnostic",
                           image_id=row.name,
                           path= PREPROCESSED_TRAINING_IMAGE_FOLDER+str(row.image_id)+".jpg",
                           labels=row['CategoryId'],
                           annotations=row['EncodedPixels'],
                           height=row['Height'], width=row['Width'],
                           img_org_id = row.image_id)

    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path'], [category_list[int(x)] for x in info['labels']]

    def load_image(self, image_id):

        return cv2.imread(self.image_info[image_id]['path'])

    def load_mask(self, image_id):
        info = self.image_info[image_id]

        mask = np.zeros((IMAGE_SIZE, IMAGE_SIZE, len(info['annotations'])), dtype=np.uint8)
        labels = []
        for m, (annotation, label) in enumerate(zip(info['annotations'], info['labels'])):
            sub_mask = np.full(info['height']*info['width'], 0, dtype=np.uint8)

            annotation = [int(x) for x in annotation.split(' ')]

            for i, start_pixel in enumerate(annotation[::2]):
                sub_mask[start_pixel: start_pixel+annotation[2*i+1]] = 1

            sub_mask = sub_mask.reshape((info['height'], info['width']), order='F')
            sub_mask = cv2.resize(sub_mask, (IMAGE_SIZE, IMAGE_SIZE), interpolation=cv2.INTER_NEAREST)

            mask[:, :, m] = sub_mask
            labels.append(int(label)+1)
        return mask, np.array(labels)

In [ ]:
# Load dataset
# Split with train = 80% samples and val = 10% and test = 10%
training_percentage = 0.8

training_set_size = int(training_percentage*len(training_df))
validation_set_size = int((0.9-training_percentage)*len(training_df))
test_set_size = int((0.9-training_percentage)*len(training_df))

train_dataset = DiagnosticDataset(training_df[:training_set_size])
train_dataset.prepare()

valid_dataset = DiagnosticDataset(training_df[training_set_size:training_set_size+validation_set_size])
valid_dataset.prepare()

test_dataset = DiagnosticDataset(training_df[training_set_size + validation_set_size:])
test_dataset.prepare()

In [ ]:
# Show test_dataset information
print("Images: {}\nClasses: {}".format(len(test_dataset.image_ids), test_dataset.class_names))

## Inspect training data

In [ ]:
def display_random_images(dataset):
    # Load and display random samples
    image_ids = np.random.choice(dataset.image_ids, 5)
    for image_id in image_ids:
        image = dataset.load_image(image_id)
        mask, class_ids = dataset.load_mask(image_id)
        visualize.display_top_masks(image, mask, class_ids, dataset.class_names, limit=8)
display_random_images(train_dataset)

In [ ]:
def display_with_image(dataset, image_id = 95):
#     image_id = np.random.choice(dataset.image_ids)

    # Load and display
    image, image_meta, class_ids, bbox, mask = modellib.load_image_gt(
            dataset, config, image_id, use_mini_mask=False)
    log("molded_image", image)
    log("mask", mask)
    visualize.display_instances(image, bbox, mask, class_ids, dataset.class_names,
                                show_bbox=False)
display_with_image(train_dataset)

## Augmentation

In [ ]:
# AUGMENTATION IMPLEMENTATION
# Image augmentation (light but constant)
augmentation = iaa.Sequential([
    iaa.OneOf([ ## geometric transform
        iaa.Affine(
            scale={"x": (0.98, 1.02), "y": (0.98, 1.04)},
            translate_percent={"x": (-0.02, 0.02), "y": (-0.04, 0.04)},
            rotate=(-2, 2),
            shear=(-1, 1),
        ),
#         iaa.PiecewiseAffine(scale=(0.001, 0.025)),
    ]),
    iaa.OneOf([ ## brightness or contrast
        iaa.Multiply((0.9, 1.1)),
        iaa.ContrastNormalization((0.9, 1.1)),
    ]),
    iaa.OneOf([ ## blur or sharpen
        iaa.GaussianBlur(sigma=(0.0, 0.1)),
        iaa.Sharpen(alpha=(0.0, 0.1)),
    ]),
])

In [ ]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
def display_image_augmentation(dataset, image_id = 95):
    # Load the image multiple times to show augmentations
    limit = 4
    ax = get_ax(rows=2, cols=limit//2)
    for i in range(limit):
        image, image_meta, class_ids, bbox, mask = modellib.load_image_gt(
            dataset, config, image_id, use_mini_mask=False, augment=False, augmentation=augmentation)
        visualize.display_instances(image, bbox, mask, class_ids,
                                    dataset.class_names, ax=ax[i//2, i % 2],
                                    show_mask=False, show_bbox=False)
display_image_augmentation(train_dataset)

## Trained Model

In [ ]:
# Call model
model = modellib.MaskRCNN(mode="inference", model_dir="",
                              config=config)

In [ ]:
# Pretrained weight
model_path = '../input/trained-weight-updated/mask_rcnn_diagnostic_0015.h5'
model.load_weights(model_path, by_name=True)

## [Run Detection](https://github.com/matterport/Mask_RCNN/blob/master/samples/nucleus/inspect_nucleus_model.ipynb)

In [ ]:
def run_detection(dataset):
    image_id = random.choice(dataset.image_ids)
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
    info = dataset.image_info[image_id]
    print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, 
                                           dataset.image_reference(image_id)))
    print("Original image shape: ", modellib.parse_image_meta(image_meta[np.newaxis,...])["original_image_shape"][0])

    # Run object detection
    results = model.detect_molded(np.expand_dims(image, 0), np.expand_dims(image_meta, 0), verbose=1)

    # Display results
    r = results[0]
    log("gt_class_id", gt_class_id)
    log("gt_bbox", gt_bbox)
    log("gt_mask", gt_mask)

    # Compute AP over range 0.5 to 0.95 and print it
    utils.compute_ap_range(gt_bbox, gt_class_id, gt_mask,
                           r['rois'], r['class_ids'], r['scores'], r['masks'],
                           verbose=1)

    visualize.display_differences(
        image,
        gt_bbox, gt_class_id, gt_mask,
        r['rois'], r['class_ids'], r['scores'], r['masks'],
        dataset.class_names, ax=get_ax(),
        show_box=False, show_mask=False,
        iou_threshold=0.5, score_threshold=0.5)
for _ in range(5):
    run_detection(test_dataset)

In [ ]:
def plot_bbox(img_id , bbox_df = org_df, normalize = True):

    img_ids = bbox_df['image_id'].values
    class_ids = bbox_df['class_id'].unique()

    label2color = {class_id:[randint(0,255) for i in range(3)] for class_id in class_ids}

    plt.figure(figsize=(20,8))
    sub_num =1

    img_id = img_id

    img_path = os.path.join(TRAIN_DIR, img_id + ".dicom")
    img = dicom2array(img_path)

    if normalize:
        # normalize
        img = exposure.equalize_adapthist(img/np.max(img))
        img = (img * 255).astype(np.uint8)

    # convert from single-channel grayscale to 3-channel RGB
    img = np.stack([img] * 3, axis=2)

    # add bounding boxes
    box_coordinates = bbox_df.loc[bbox_df['image_id'] == img_id, ['x_min', 'y_min', 'x_max', 'y_max']].values
    labels = bbox_df.loc[bbox_df['image_id'] == img_id, ['class_id']].values.squeeze()
    if not labels.shape:
        labels = np.expand_dims(labels, axis =0)

    for label_id, box in zip(labels, box_coordinates):
        color = label2color[label_id]
        img_bbox = cv2.rectangle(
            img,
            (int(box[0]), int(box[1])),
            (int(box[2]), int(box[3])),
            color = color, thickness= 8
        )
        # add labels
        cv2.putText(img_bbox, str(label_id), (int(box[0]), int(box[1])), cv2.FONT_HERSHEY_SIMPLEX, 5, (36,255,12), 5)

    plt.subplot(1,3,sub_num)
    sub_num += 1
    plt.imshow(img_bbox, cmap = 'gray')
    plt.title('Finding contains in image')

    plt.show()

In [ ]:
def dicom2array(path, voi_lut=True, fix_monochrome=True):
    dicom = pydicom.read_file(path)
    # VOI LUT (if available by DICOM device) is used to
    # transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data

    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)

    return data

In [ ]:
# Display original
def display_test_result(dataset):
    image_id = random.choice(dataset.image_ids)
    
#     print(image_id)
    # Display original Dicom
    
    print('GT - Dicom')
    plot_bbox(img_id = dataset.image_info[image_id]['img_org_id'])

    # Display original in training form

    original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset, config, 
                               image_id, use_mini_mask=False)
    
    print('GT')
    visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                                dataset.class_names, figsize=(5, 5))
    
    # Display test prediction

    results = model.detect([original_image], verbose=0)
    r = results[0]
    
    print('Predict')

    visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                                dataset.class_names, r['scores'], figsize=(5, 5))

In [ ]:
for _ in range(5):
    display_test_result(dataset = test_dataset)

## Submission

In [ ]:
def find_anomalies(dicom_image):

    image_dimensions = dicom_image.shape

    resized_img = cv2.resize(dicom_image, (IMAGE_SIZE,IMAGE_SIZE), interpolation = cv2.INTER_AREA)
    saved_filename = TEMP_TEST_FOLDER+"temp_image.jpg"
    
    cv2.imwrite(saved_filename, resized_img) 
    
    img = cv2.imread(saved_filename)

    result = model.detect([img])
    r = result[0]
    
    if r['masks'].size > 0:
        y_scale = image_dimensions[0]/IMAGE_SIZE
        x_scale = image_dimensions[1]/IMAGE_SIZE
        rois = (r['rois'] * [y_scale, x_scale, y_scale, x_scale]).astype(int)
        
    else:
        rois = r['rois']
        
    return rois, r['class_ids'], r['scores']

In [ ]:
selected_classes_dict = {}
key_value = 0
for _ in train_dataset.class_names:
#     print(key_value)
    selected_classes_dict[str(key_value)] = key_value 
    key_value += 1
# selected_classes_dict

In [ ]:
def submission_results(visualize_mask = True, num_test = 5):
    results = []
    test_file_list = os.listdir(TEST_DIR)


    for image_file_name in tqdm(test_file_list[:num_test]):

        dicom_image = dicom2array(TEST_DIR + '/' + image_file_name)
        image_dimensions = dicom_image.shape

        # extracrt results
        bbox_list, class_list, confidence_list = find_anomalies(dicom_image)

        # convert from single-channel grayscale to 3-channel RGB
        img = np.stack([dicom_image] * 3, axis=2)
        resized_img = cv2.resize(img, (IMAGE_SIZE,IMAGE_SIZE), interpolation = cv2.INTER_AREA)

        # visualize
        result = model.detect([resized_img])
        r = result[0]

        if visualize_mask:
            visualize.display_instances(resized_img, r['rois'], r['masks'], r['class_ids'], 
                                        train_dataset.class_names, r['scores'], show_mask = False,
                                       figsize=(5,5))

        # found abnormalities
        prediction_string = ""

        if len(bbox_list) > 0:

            for bbox, class_id, confidence in zip(bbox_list, class_list, confidence_list):

                # Convert to submission class id
                for key, value in selected_classes_dict.items():
                    if class_id == int(key):
                        class_id_correct = value -1 # minus 1 because start from 0 for abnormal cases

                confidence_score = str(round(confidence,3))

                #organise the bbox into xmin, ymin, xmax, ymax
                ymin = bbox[2]
                ymax = bbox[0]
                xmin = bbox[1]
                xmax = bbox[3]
                if xmin > xmax:
                    xmin, xmax = xmax, xmin
                if ymin > ymax:
                    ymin, ymax = ymax, ymin

                prediction_string += "{} {} {} {} {} {} ".format(class_id_correct, confidence_score, xmin, ymin, xmax, ymax)

            results.append({"image_id":image_file_name.replace(".dicom",""), "PredictionString":prediction_string.strip()})

        else:
            results.append({"image_id":image_file_name.replace(".dicom",""), "PredictionString":"14 1.0 0 0 1 1"})
    submission_df = pd.DataFrame(results)
    return submission_df

In [ ]:
submission_results(visualize_mask = True, num_test = 5)

In [ ]:
submission_df = submission_results(visualize_mask = False, num_test = 3000)

In [ ]:
submission_df.to_csv('submission.csv', index = False)